<a href="https://colab.research.google.com/github/Michael-yousuf123/next-word-prediction/blob/master/notebook/next_word_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/My Drive/medium_data.csv")

In [4]:
df.head()

,id,url,title,subtitle,image,claps,responses,reading_time,publication,date
0,1,https://towardsdatascience.com/a-beginners-gui...,A Beginner’s Guide to Word Embedding with Gens...,NaN,1.png,850,8,8,Towards Data Science,2019-05-30
1,2,https://towardsdatascience.com/hands-on-graph-...,Hands-on Graph Neural Networks with PyTorch & ...,NaN,2.png,1100,11,9,Towards Data Science,2019-05-30
2,3,https://towardsdatascience.com/how-to-use-ggpl...,How to Use ggplot2 in Python,A Grammar of Graphics for Python,3.png,767,1,5,Towards Data Science,2019-05-30
3,4,https://towardsdatascience.com/databricks-how-...,Databricks: How to Save Files in CSV on Your L...,When I work on Python projects dealing…,4.jpeg,354,0,4,Towards Data Science,2019-05-30
4,5,https://towardsdatascience.com/a-step-by-step-...,A Step-by-Step Implementation of Gradient Desc...,One example of building neural…,5.jpeg,211,3,4,Towards Data Science,2019-05-30


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6508 entries, 0 to 6507
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            6508 non-null   int64 
 1   url           6508 non-null   object
 2   title         6508 non-null   object
 3   subtitle      3479 non-null   object
 4   image         6361 non-null   object
 5   claps         6508 non-null   int64 
 6   responses     6508 non-null   object
 7   reading_time  6508 non-null   int64 
 8   publication   6508 non-null   object
 9   date          6508 non-null   object
dtypes: int64(3), object(7)
memory usage: 508.6+ KB


In [6]:
df = df['title']

In [7]:
df

0       A Beginner’s Guide to Word Embedding with Gens...
1       Hands-on Graph Neural Networks with PyTorch & ...
2                            How to Use ggplot2 in Python
3       Databricks: How to Save Files in CSV on Your L...
4       A Step-by-Step Implementation of Gradient Desc...
                              ...                        
6503    “We” vs “I” — How Should You Talk About Yourse...
6504                     How Donald Trump Markets Himself
6505        Content and Marketing Beyond Mass Consumption
6506    5 Questions All Copywriters Should Ask Clients...
6507               How To Write a Good Business Blog Post
Name: title, Length: 6508, dtype: object

In [ ]:
df.shape

In [20]:
import re 
import string 
import nltk 
nltk.download("stopwords")
nltk.download('punkt')
from nltk.corpus import stopwords, words  
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [21]:
stop_words = list(stopwords.words('english'))

In [22]:
def clean_text(text):
  """function to clean data"""
  text = text.lower() 
  text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", text).split())
  text = re.sub(r'\@\w+|\#\w+|\d+', '', text)
  punct = str.maketrans('', '', string.punctuation+string.digits)
  text = text.translate(punct)
  # tokenizing words and removing stop words from the tweet text
  tokens = word_tokenize(text)  
  filtered_words = [w for w in tokens if w not in stop_words] 
  text = " ".join(filtered_words)
  return text

In [23]:
tokens = df.apply(clean_text)

In [24]:
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

0      beginner ’ guide word embedding gensim wordvec...
1      handson graph neural networks pytorch pytorch ...
2                                      use ggplot python
3               databricks save files csv local computer
4      stepbystep implementation gradient descent bac...
                             ...                        
195                             predicting vs explaining
196           machine learning models degrade production
197                          implementation rnn lstm gru
198                              autoencoders vs pca use
199     introduction recurrent neural networks beginners
Name: title, Length: 200, dtype: object
Total Tokens: 6508
Unique Tokens: 6492


In [25]:
# organize into sequences of tokens
length = 1 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 6506


In [26]:
sequences

['beginner ’ guide word embedding gensim wordvec model handson graph neural networks pytorch pytorch geometric',
 'handson graph neural networks pytorch pytorch geometric use ggplot python',
 'use ggplot python databricks save files csv local computer',
 'databricks save files csv local computer stepbystep implementation gradient descent backpropagation',
 'stepbystep implementation gradient descent backpropagation easy introduction sql data scientists',
 'easy introduction sql data scientists hypothesis testing visualized',
 'hypothesis testing visualized introduction latent matrix factorization recommender systems',
 'introduction latent matrix factorization recommender systems candidate best twitter',
 'candidate best twitter ai model understanding easy',
 'ai model understanding easy em classmarkupem markuphemwhat learned twotime kaggle grandmaster abhishek thakurem',
 'em classmarkupem markuphemwhat learned twotime kaggle grandmaster abhishek thakurem making dota bot using ml',
 '

In [27]:
from numpy import array
from keras.callbacks import CSVLogger
import numpy as np
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Embedding
import io

In [29]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
sequences = tokenizer.texts_to_sequences(sequences)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [30]:
vocab_size

8258

In [31]:
sequences

[[423, 1, 28, 365, 1665, 3472, 8257, 48, 3475, 600, 35, 52, 241, 241, 1666],
 [3475, 600, 35, 52, 241, 241, 1666, 25, 3476, 13],
 [25, 3476, 13, 3477, 177, 1339, 2254, 785, 185],
 [3477, 177, 1339, 2254, 785, 185, 786, 366, 1667, 1095, 1340],
 [786, 366, 1667, 1095, 1340, 121, 66, 337, 3, 255],
 [121, 66, 337, 3, 255, 1341, 207, 3478],
 [1341, 207, 3478, 66, 3479, 601, 2255, 602, 223],
 [66, 3479, 601, 2255, 602, 223, 1668, 31, 277],
 [1668, 31, 277, 12, 48, 112, 121],
 [12, 48, 112, 121, 603, 338, 3480, 49, 3481, 678, 3482, 3483, 3484],
 [603, 338, 3480, 49, 3481, 678, 3482, 3483, 3484, 70, 2256, 481, 8, 191],
 [70, 2256, 481, 8, 191, 57, 75, 679, 1669, 3485],
 [57, 75, 679, 1669, 3485, 424, 108],
 [424, 108, 913, 2257, 142, 118],
 [913, 2257, 142, 118, 535, 2258, 425],
 [535, 2258, 425, 160, 10, 3486, 3, 1670],
 [160, 10, 3486, 3, 1670, 3487, 2259, 2260, 2261, 2260, 602],
 [3487, 2259, 2260, 2261, 2260, 602, 3, 15, 256, 222],
 [3, 15, 256, 222, 680, 367, 681, 1671],
 [680, 367, 681, 

In [32]:
length = 2

In [33]:
print(len(sequences))
print(sequences[0][0])

6506
423


In [34]:
sequences = array(sequences)
sequences

<ipython-input-34-82b1dc3cafc8>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sequences = array(sequences)


array([list([423, 1, 28, 365, 1665, 3472, 8257, 48, 3475, 600, 35, 52, 241, 241, 1666]),
       list([3475, 600, 35, 52, 241, 241, 1666, 25, 3476, 13]),
       list([25, 3476, 13, 3477, 177, 1339, 2254, 785, 185]), ...,
       list([18, 60, 18, 76, 2, 259, 319, 8254, 660, 361]),
       list([8254, 660, 361, 84, 27, 324, 8255, 8256]),
       list([84, 27, 324, 8255, 8256, 222, 3473, 240, 480, 1094, 3474, 599])],
      dtype=object)

In [35]:
c=1
h=0
while(c!=0):
    limit = sequences.size
    d =[]
    c=0
    for i in range(0 , limit):
        l=len(sequences[i])
        if(l!=length):
            print(sequences[i])
            sequences = np.delete(sequences, i, 0)
            
            c=1
            print("error at ", i)
            print("Deleting :_________________________ ") 
            h = h+1
            break        
print("Total deletion: ", h) 

Streaming output truncated to the last 5000 lines.
Deleting :_________________________ 
[806, 3041, 395, 2677, 60, 2323, 116, 3386, 60, 7043]
error at  0
Deleting :_________________________ 
[2677, 60, 2323, 116, 3386, 60, 7043, 167, 3099, 7044, 938, 3, 15, 117]
error at  0
Deleting :_________________________ 
[167, 3099, 7044, 938, 3, 15, 117, 7045, 7046, 7047, 62]
error at  0
Deleting :_________________________ 
[7045, 7046, 7047, 62, 213, 109, 251]
error at  0
Deleting :_________________________ 
[213, 109, 251, 1993, 100, 7048, 834]
error at  0
Deleting :_________________________ 
[1993, 100, 7048, 834, 197, 625, 31, 732, 11, 2413]
error at  0
Deleting :_________________________ 
[197, 625, 31, 732, 11, 2413, 86, 203, 56, 7049, 45, 7050, 596, 473, 2, 11, 952]
error at  0
Deleting :_________________________ 
[86, 203, 56, 7049, 45, 7050, 596, 473, 2, 11, 952, 4, 5, 7051, 36, 88, 7052, 2, 11, 1828, 981]
error at  0
Deleting :_________________________ 
[4, 5, 7051, 36, 88, 7052, 2, 11

In [36]:
sequences = np.array(sequences.tolist())
sequences 

array([[8079, 8080]])

In [37]:
sequences.shape

(1, 2)

In [38]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [39]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(GRU(100, return_sequences=True))
model.add(GRU(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
csv_logger = CSVLogger('training_1_1.log')
model.fit(X, y, batch_size=500, epochs=1000, callbacks=[csv_logger])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 50)             412900    
                                                                 
 gru (GRU)                   (None, 1, 100)            45600     
                                                                 
 gru_1 (GRU)                 (None, 100)               60600     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 8258)              834058    
                                                                 
Total params: 1,363,258
Trainable params: 1,363,258
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000
1/1 [=======================

In [40]:
# save the model to file
model.save('/content/drive/My Drive/final/model.h5')

# save the tokenizer
dump(tokenizer, open('/content/drive/My Drive/final/tokenizer.pkl', 'wb'))